In [7]:
import os
import sys
import csv
import base64
import time
from pathlib import Path

import numpy as np

import json
import cv2 as cv
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm

tiny,fast = True,False

TINY_IMG_NUM = 512
FAST_IMG_NUM = 5000

csv.field_size_limit(sys.maxsize)
FIELDNAMES = ["img_id", "img_h", "img_w", "objects_id", "objects_conf",
              "attrs_id", "attrs_conf", "num_boxes", "boxes", "features"]
SPLIT2NAME = {
    'train': 'train2014',
    'valid': 'val2014',
    'minival': 'val2014',
    'nominival': 'val2014',
    'test': 'test2015',
}


## Loading tsv

In [8]:
split = 'minival'

home = str(Path.home())
MSCOCO_IMGFEAT_ROOT = home + '/data/mscoco_imgfeat/'

In [27]:
fname = MSCOCO_IMGFEAT_ROOT+ '%s_obj36.tsv' % SPLIT2NAME[split]

In [28]:
fname

'/home/u37216/data/mscoco_imgfeat/val2014_obj36.tsv'

In [29]:
import pandas as pd

In [ ]:
with open(fname) as f:
        reader = csv.DictReader(f, FIELDNAMES, delimiter="\t")
        for i, item in tqdm(enumerate(reader)):

            for key in ['img_h', 'img_w', 'num_boxes']:
                item[key] = int(item[key])
            
            boxes = item['num_boxes']
            decode_config = [
                ('objects_id', (boxes, ), np.int64),
                ('objects_conf', (boxes, ), np.float32),
                ('attrs_id', (boxes, ), np.int64),
                ('attrs_conf', (boxes, ), np.float32),
                ('boxes', (boxes, 4), np.float32),
                ('features', (boxes, -1), np.float32),
            ]
            for key, shape, dtype in decode_config:
                item[key] = np.frombuffer(base64.b64decode(item[key]), dtype=dtype)
                item[key] = item[key].reshape(shape)
                item[key].setflags(write=False)

            data.append(item)
    return data

In [43]:
t = pd.read_csv(fname, delimiter="\t",chunksize=1)

In [44]:
for i, item in enumerate(t):
    print(item)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



    COCO_val2014_000000568281  480  640  \
56  COCO_val2014_000000111032  400  500   

   KwAAAAAAAAD4AAAAAAAAAMgAAAAAAAAA+AAAAAAAAABhAwAAAAAAAGoAAAAAAAAA8AAAAAAAAAAhAgAAAAAAAE8DAAAAAAAAfQAAAAAAAADnAAAAAAAAAOcAAAAAAAAAagAAAAAAAACxAwAAAAAAADoBAAAAAAAAyAAAAAAAAABqAAAAAAAAACECAAAAAAAATwMAAAAAAACxAwAAAAAAAP0CAAAAAAAA8AAAAAAAAAAuAAAAAAAAAGICAAAAAAAA8gEAAAAAAAAGAQAAAAAAAGoAAAAAAAAA+AAAAAAAAADyAAAAAAAAACECAAAAAAAAagAAAAAAAADIAAAAAAAAAGICAAAAAAAABgEAAAAAAAD4AAAAAAAAAGEDAAAAAAAA  \
56  agAAAAAAAACBBAAAAAAAAIkAAAAAAAAA2gEAAAAAAACBBA...                                                                                                                                                                                                                                                                                                                                                 

   wgt/P46NWT9VYk8/mUBEP0BEND9wZTI/CcUqP8r6JD/WVRM/I5wCPyoH8T7vluc+14TLPvYavz4nLrA+RfmtPjGGpj6jsY8+VmaOPnJ1lz6ITIM+duGAPu+Ncz6BiHA

KeyboardInterrupt: 